In [1]:
%matplotlib inline
import scipy
import numpy as np
import pandas as pd
import pandas.io.formats.style
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from IPython.display import HTML
import linearmodels as lm
from linearmodels import PanelOLS
import warnings
warnings.filterwarnings("ignore")
import math
from linearmodels.panel import compare
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm_api


In [2]:
# path="/Users/mengxi/Documents/GitHub/research_module/data/"
# data=pd.read_stata(path+"county_clean/total_and_by_industry_data_county.dta")


path="/Users/effie/Desktop/term3/research_module/data/"
data=pd.read_stata(path+"county_clean/119_total_and_by_industry_data_county.dta")
data=data.drop(columns=['index'])

data=data.rename(columns={"ln_gdp": "gdp_growth_100"})

data


,year,Gebietseinheit,EU_Code,Regional_key,Land,resident,gdp_growth_100,gdp_level,gdp_per_inhabitant,gdp_per_employee,...,toas_ratio,emp_agr_weight,emp_man_weight,emp_ser_weight,gva_agr_weight,gva_man_weight,gva_ser_weight,vow_agr_weight,vow_man_weight,vow_ser_weight
0,2005,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,08111,BW,574.627014,-2.595548,39624.353,68957.0,83804.0,...,1.0,0.002310,0.210602,0.787089,0.000741,0.321257,0.678002,0.002739,0.219986,0.777276
1,2005,"Böblingen, Landkreis",DE112,08115,BW,366.231995,-10.258647,12993.051,35478.0,61177.0,...,1.0,0.004826,0.412613,0.582561,0.001957,0.435541,0.562501,0.006725,0.435416,0.557859
2,2005,"Esslingen, Landkreis",DE113,08116,BW,504.429993,0.212459,15237.577,30208.0,61109.0,...,1.0,0.006164,0.369497,0.624339,0.002130,0.393602,0.604268,0.007894,0.389075,0.603031
3,2005,"Göppingen, Landkreis",DE114,08117,BW,254.494995,-1.309602,6077.935,23882.0,54336.0,...,1.0,0.012873,0.394026,0.593100,0.005486,0.417749,0.576766,0.019193,0.417092,0.563714
4,2005,"Ludwigsburg, Landkreis",DE115,08118,BW,505.869995,2.257281,16209.134,32042.0,68900.0,...,1.0,0.014138,0.338531,0.647332,0.004291,0.380648,0.615061,0.018165,0.357240,0.624595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2403,2012,Saale-Holzland-Kreis,DEG0J,16074,TH,84.480003,2.386800,1734.716,20534.0,48577.0,...,0.0,0.042256,0.362381,0.595363,0.028101,0.363217,0.608683,NaN,NaN,NaN
2404,2012,"Greiz, Landkreis",DEG0L,16076,TH,103.880997,2.040082,1854.354,17851.0,46776.0,...,0.0,0.037358,0.373862,0.588780,0.031252,0.365997,0.602751,NaN,NaN,NaN
2405,2012,"Altenburger Land, Landkreis",DEG0M,16077,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,...,1.0,0.028015,0.338902,0.633083,0.023566,0.340479,0.635954,NaN,NaN,NaN
2406,2012,"Eisenach, Kreisfreie Stadt",DEG0N,16056,TH,NaN,NaN,NaN,NaN,NaN,...,0.0,0.002846,0.296681,0.700473,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data["No_ratio"].describe()

count    2408.000000
mean        0.286297
std         0.338936
min         0.000000
25%         0.000000
50%         0.166667
75%         0.500000
max         1.000000
Name: No_ratio, dtype: float64

In [4]:
data["toas_ratio"].describe()

count    2408.000000
mean        0.627292
std         0.475227
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.146924
Name: toas_ratio, dtype: float64

In [5]:
data['Land'].value_counts()

BY    560
NW    392
BW    296
NI    208
HE    184
RP    160
TH    104
SN     96
ST     96
SH     88
BB     88
MV     64
SL     40
HB     16
BE      8
HH      8
Name: Land, dtype: int64

In [6]:
query_data = data.query('No_ratio>0.286')
data.loc[:,'county_affected_no'] = data.index.map(lambda x : 1 if x in query_data.index else 0)

query_data = data.query('toas_ratio>0.627')
data.loc[:,'county_affected_toas'] = data.index.map(lambda x : 1 if x in query_data.index else 0)

query_data = data.query('year>2008')
data.loc[:,'year_post'] = data.index.map(lambda x : 1 if x in query_data.index else 0)

data["affected_after_toas"] = data.county_affected_toas*data.year_post
data["affected_after_toas_ratio"]=data.toas_ratio*data.year_post


data

,year,Gebietseinheit,EU_Code,Regional_key,Land,resident,gdp_growth_100,gdp_level,gdp_per_inhabitant,gdp_per_employee,...,gva_man_weight,gva_ser_weight,vow_agr_weight,vow_man_weight,vow_ser_weight,county_affected_no,county_affected_toas,year_post,affected_after_toas,affected_after_toas_ratio
0,2005,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,08111,BW,574.627014,-2.595548,39624.353,68957.0,83804.0,...,0.321257,0.678002,0.002739,0.219986,0.777276,1,1,0,0,0.0
1,2005,"Böblingen, Landkreis",DE112,08115,BW,366.231995,-10.258647,12993.051,35478.0,61177.0,...,0.435541,0.562501,0.006725,0.435416,0.557859,0,1,0,0,0.0
2,2005,"Esslingen, Landkreis",DE113,08116,BW,504.429993,0.212459,15237.577,30208.0,61109.0,...,0.393602,0.604268,0.007894,0.389075,0.603031,0,1,0,0,0.0
3,2005,"Göppingen, Landkreis",DE114,08117,BW,254.494995,-1.309602,6077.935,23882.0,54336.0,...,0.417749,0.576766,0.019193,0.417092,0.563714,1,1,0,0,0.0
4,2005,"Ludwigsburg, Landkreis",DE115,08118,BW,505.869995,2.257281,16209.134,32042.0,68900.0,...,0.380648,0.615061,0.018165,0.357240,0.624595,0,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2403,2012,Saale-Holzland-Kreis,DEG0J,16074,TH,84.480003,2.386800,1734.716,20534.0,48577.0,...,0.363217,0.608683,NaN,NaN,NaN,0,0,1,0,0.0
2404,2012,"Greiz, Landkreis",DEG0L,16076,TH,103.880997,2.040082,1854.354,17851.0,46776.0,...,0.365997,0.602751,NaN,NaN,NaN,0,0,1,0,0.0
2405,2012,"Altenburger Land, Landkreis",DEG0M,16077,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,...,0.340479,0.635954,NaN,NaN,NaN,1,1,1,1,1.0
2406,2012,"Eisenach, Kreisfreie Stadt",DEG0N,16056,TH,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0.0


In [7]:
data['county_affected_toas'].value_counts()

1    1512
0     896
Name: county_affected_toas, dtype: int64

In [8]:
data["affected_after_toas_ratio"].value_counts()

0.000000    1612
1.000000     704
0.862076       4
0.991118       4
0.014877       4
0.084142       4
0.705943       4
0.194287       4
0.167133       4
0.996335       4
0.873121       4
0.151608       4
0.102946       4
0.717289       4
0.240669       4
0.999252       4
0.836010       4
1.146924       4
1.000154       4
0.427839       4
0.167701       4
0.840457       4
0.028159       4
0.632758       4
0.634103       4
Name: affected_after_toas_ratio, dtype: int64

In [9]:


data["ln_res"]=np.log(data["resident"]*1000)

data["ln_gdp_level"] = np.log(data.gdp_level*1000000)
data["ln_gdp_per_inhabitant"] = np.log(data.gdp_per_inhabitant)
data["ln_gdp_per_employee"] = np.log(data.gdp_per_employee)
data["ln_gdp_per_hour_per_employed"] = np.log(data.gdp_per_hour_per_employed)



data["ln_gva_level"] = np.log(data.gva_level*1000000)
data["ln_gva_agr"] = np.log(data.gva_agr*1000000)
data["ln_gva_man"] = np.log(data.gva_man*1000000)
data["ln_gva_ser"] = np.log(data.gva_ser*1000000)
data["ln_gva_ser_fin"] = np.log(data.gva_ser_fin*1000000)
data["ln_gva_ser_trade"] = np.log(data.gva_ser_trade*1000000)
data["ln_gva_ser_public"] = np.log(data.gva_ser_public*1000000)

data["ln_emp"]=np.log(data["employee"]*1000)

data["ln_emp_ser_fin"] = np.log(data.emp_ser_fin*1000)
data["ln_emp_ser_trade"] = np.log(data.emp_ser_trade*1000)
data["ln_emp_ser_public"] = np.log(data.emp_ser_public*1000)
data["ln_emp_agr"]=np.log(data["emp_agr"]*1000)
data["ln_emp_man"]=np.log(data["emp_man"]*1000)
data["ln_emp_ser"]=np.log(data["emp_ser"]*1000)

data["ln_vow"]=np.log(data["volume_of_work"]*1000000)

data["ln_vow_ser_fin"] = np.log(data.vow_ser_fin*1000000)
data["ln_vow_ser_trade"] = np.log(data.vow_ser_trade*1000000)
data["ln_vow_ser_public"] = np.log(data.vow_ser_public*1000000)
data["ln_vow_agr"]=np.log(data["vow_agr"]*1000000)
data["ln_vow_man"]=np.log(data["vow_man"]*1000000)
data["ln_vow_ser"]=np.log(data["vow_ser"]*1000000)

data["ln_fixed_total"] = np.log(data.fixed_total*1000000)
data["ln_fixed_ser_fin"] = np.log(data.fixed_ser_fin*1000000)
data["ln_fixed_ser_fin"] = np.log(data.fixed_ser_fin*1000000)
data["ln_fixed_ser_trade"] = np.log(data.fixed_ser_trade*1000000)
data["ln_fixed_ser_public"] = np.log(data.fixed_ser_public*1000000)
data["ln_fixed_agr"]=np.log(data["fixed_agr"]*1000000)
data["ln_fixed_man"]=np.log(data["fixed_man"]*1000000)
data["ln_fixed_ser"]=np.log(data["fixed_ser"]*1000000)

data["gdp_growth"]=data["gdp_growth_100"]/100


df_fe = data.set_index(['Regional_key','year'])
df_fe

,,Gebietseinheit,EU_Code,Land,resident,gdp_growth_100,gdp_level,gdp_per_inhabitant,gdp_per_employee,gdp_per_hour_per_employed,ln_gva,...,ln_vow_man,ln_vow_ser,ln_fixed_total,ln_fixed_ser_fin,ln_fixed_ser_trade,ln_fixed_ser_public,ln_fixed_agr,ln_fixed_man,ln_fixed_ser,gdp_growth
Regional_key,year,,,,,,,,,,,,,,,,,,,,,
08111,2005,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,BW,574.627014,-2.595548,39624.353,68957.0,83804.0,57.240002,-2.754354,...,18.841323,20.103557,24.923215,24.056016,22.830785,22.974705,20.435918,23.728365,24.546337,-0.025955
08115,2005,"Böblingen, Landkreis",DE112,BW,366.231995,-10.258647,12993.051,35478.0,61177.0,42.860001,-10.404957,...,18.698373,18.946178,24.923215,24.056016,22.830785,22.974705,20.435918,23.728365,24.546337,-0.102586
08116,2005,"Esslingen, Landkreis",DE113,BW,504.429993,0.212459,15237.577,30208.0,61109.0,42.930000,0.049075,...,18.743547,19.181744,24.923215,24.056016,22.830785,22.974705,20.435918,23.728365,24.546337,0.002125
08117,2005,"Göppingen, Landkreis",DE114,BW,254.494995,-1.309602,6077.935,23882.0,54336.0,38.509998,-1.470492,...,18.002480,18.303720,24.923215,24.056016,22.830785,22.974705,20.435918,23.728365,24.546337,-0.013096
08118,2005,"Ludwigsburg, Landkreis",DE115,BW,505.869995,2.257281,16209.134,32042.0,68900.0,48.599998,2.090561,...,18.595810,19.154504,24.923215,24.056016,22.830785,22.974705,20.435918,23.728365,24.546337,0.022573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16074,2012,Saale-Holzland-Kreis,DEG0J,TH,84.480003,2.386800,1734.716,20534.0,48577.0,NaN,2.438582,...,NaN,NaN,23.172179,22.076258,21.326727,21.618260,19.454235,21.870640,22.820639,0.023868
16076,2012,"Greiz, Landkreis",DEG0L,TH,103.880997,2.040082,1854.354,17851.0,46776.0,NaN,2.091659,...,NaN,NaN,23.172179,22.076258,21.326727,21.618260,19.454235,21.870640,22.820639,0.020401
16077,2012,"Altenburger Land, Landkreis",DEG0M,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,NaN,-1.566608,...,NaN,NaN,23.172179,22.076258,21.326727,21.618260,19.454235,21.870640,22.820639,-0.016161


<a class="anchor" id="8. Difference in differences implementation"></a>

---
# gdp


##  gdp - toas

In [10]:
# use ln_gdp_per_hour_per_employed  or just gdp_per_hour_per_employed in with control did regression??
exog_vars = ['affected_after_toas_ratio',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=True,time_effects=True)
res_fe_ln_gdp_level = mod.fit(cov_type = 'clustered',cluster_entity=False)




exog_vars = ['affected_after_toas_ratio',"gva_agr_weight","gva_man_weight","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=True,time_effects=True)
res_fe_ln_gdp_per_inhabitant = mod.fit(cov_type = 'clustered',cluster_entity=False)




exog_vars = ['affected_after_toas_ratio',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=True,time_effects=True)
res_fe_ln_gdp_per_employee = mod.fit(cov_type = 'clustered',cluster_entity=False)



exog_vars = ['affected_after_toas_ratio',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_hour_per_employed,exog,entity_effects=True,time_effects=True)
res_fe_ln_gdp_per_hour_per_employed = mod.fit(cov_type = 'clustered',cluster_entity=False)


# gdp growth
exog_vars = ['affected_after_toas_ratio',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_growth,exog,entity_effects=True,time_effects=True)
res_fe_gdp_growth = mod.fit(cov_type = 'clustered',cluster_entity=False)





In [11]:
print(compare({"ln_gdp_level": res_fe_ln_gdp_level, "ln_gdp_per_inhabitant": res_fe_ln_gdp_per_inhabitant, "ln_gdp_per_employee": res_fe_ln_gdp_per_employee}, stars = True, precision = "std_errors"))

                                        Model Comparison                                        
                                  ln_gdp_level     ln_gdp_per_inhabitant     ln_gdp_per_employee
------------------------------------------------------------------------------------------------
Dep. Variable                     ln_gdp_level     ln_gdp_per_inhabitant     ln_gdp_per_employee
Estimator                             PanelOLS                  PanelOLS                PanelOLS
No. Observations                          2376                      2376                    2376
Cov. Est.                            Clustered                 Clustered               Clustered
R-squared                               0.4602                    0.4576                  0.4531
R-Squared (Within)                      0.3089                    0.2859                  0.4760
R-Squared (Between)                     0.5059                   -0.1544                 -0.9145
R-Squared (Overall)           

In [12]:
print(compare({"ln_gdp_per_hour_per_employed": res_fe_ln_gdp_per_hour_per_employed, "gdp_growth": res_fe_gdp_growth,}, stars = True, precision = "std_errors"))

                               Model Comparison                              
                                  ln_gdp_per_hour_per_employed     gdp_growth
-----------------------------------------------------------------------------
Dep. Variable                     ln_gdp_per_hour_per_employed     gdp_growth
Estimator                                             PanelOLS       PanelOLS
No. Observations                                          2288           2376
Cov. Est.                                            Clustered      Clustered
R-squared                                               0.4420         0.1382
R-Squared (Within)                                      0.4155         0.1997
R-Squared (Between)                                    -1.2485        -90.073
R-Squared (Overall)                                    -1.0971        -4.7311
F-statistic                                             315.22         66.290
P-value (F-stat)                                        0.0000  

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# gva

##  gva - toas

In [13]:


exog_vars = ['affected_after_toas_ratio',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_level = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas_ratio',"gva_agr_weight","ln_res","ln_fixed_agr"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_agr = mod.fit(cov_type = 'clustered',cluster_entity=False)




exog_vars = ['affected_after_toas_ratio',"gva_man_weight","ln_res","ln_fixed_man"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_man = mod.fit(cov_type = 'clustered',cluster_entity=False)



exog_vars = ['affected_after_toas_ratio',"gva_ser_weight","ln_res","ln_fixed_ser"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_ser = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"ln_gva_level": res_fe_ln_gva_level, "ln_gva_agr": res_fe_ln_gva_agr,"ln_gva_man": res_fe_ln_gva_man, "ln_gva_ser": res_fe_ln_gva_ser}, stars = True, precision = "std_errors"))

                                      Model Comparison                                     
                                  ln_gva_level     ln_gva_agr     ln_gva_man     ln_gva_ser
-------------------------------------------------------------------------------------------
Dep. Variable                     ln_gva_level     ln_gva_agr     ln_gva_man     ln_gva_ser
Estimator                             PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                          2376           2376           2376           2376
Cov. Est.                            Clustered      Clustered      Clustered      Clustered
R-squared                               0.4602         0.2560         0.8809         0.1364
R-Squared (Within)                      0.3245         0.4339         0.6917         0.0624
R-Squared (Between)                     0.5059         0.3475         0.5377         0.5653
R-Squared (Overall)                     0.5045         0.3494         0.5407    

In [14]:


exog_vars = ['affected_after_toas_ratio',"emp_ser_weight","ln_res","ln_fixed_ser_fin"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_ser_fin = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas_ratio',"emp_ser_weight","ln_res","ln_fixed_ser_trade"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_ser_trade = mod.fit(cov_type = 'clustered',cluster_entity=False)



exog_vars = ['affected_after_toas_ratio',"emp_ser_weight","ln_res","ln_fixed_ser_public"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_ln_gva_ser_public = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"ln_gva_ser_fin": res_fe_ln_gva_ser_fin, "ln_gva_ser_trade": res_fe_ln_gva_ser_trade, "ln_gva_ser_public": res_fe_ln_gva_ser_public,}, stars = True, precision = "std_errors"))


                                      Model Comparison                                     
                                  ln_gva_ser_fin     ln_gva_ser_trade     ln_gva_ser_public
-------------------------------------------------------------------------------------------
Dep. Variable                     ln_gva_ser_fin     ln_gva_ser_trade     ln_gva_ser_public
Estimator                               PanelOLS             PanelOLS              PanelOLS
No. Observations                            2392                 2392                  2392
Cov. Est.                              Clustered            Clustered             Clustered
R-squared                                 0.0311               0.1362                0.1225
R-Squared (Within)                        0.0397               0.0368               -0.1494
R-Squared (Between)                       0.3638               0.6983                0.6506
R-Squared (Overall)                       0.3612               0.6954           

## empl toas

In [15]:
exog_vars = ['affected_after_toas_ratio','ln_res',"emp_agr_weight","emp_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=True,time_effects=True)
res_fe_1 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas_ratio','ln_res',"emp_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas_ratio','ln_res',"emp_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=True,time_effects=True)
res_fe_3 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas_ratio','ln_res',"emp_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

                                     Model Comparison                                    
                                         (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------------
Dep. Variable                         ln_emp     ln_emp_agr     ln_emp_man     ln_emp_ser
Estimator                           PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                        2392           2392           2392           2392
Cov. Est.                          Clustered      Clustered      Clustered      Clustered
R-squared                             0.3005         0.1831         0.6550         0.4719
R-Squared (Within)                   -0.1006         0.2745         0.3559         0.1857
R-Squared (Between)                   0.7876         0.3895         0.6179         0.7845
R-Squared (Overall)                   0.7862         0.3890         0.6172         0.7834
F-statisti

In [16]:
exog_vars = ['affected_after_toas_ratio','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas_ratio','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas_ratio','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3}, stars = True, precision = "std_errors"))

                               Model Comparison                               
                                         (1)                (2)            (3)
------------------------------------------------------------------------------
Dep. Variable                         ln_emp     ln_emp_ser_fin     ln_emp_man
Estimator                           PanelOLS           PanelOLS       PanelOLS
No. Observations                        2392               2392           2392
Cov. Est.                          Clustered          Clustered      Clustered
R-squared                             0.3005             0.1184         0.6550
R-Squared (Within)                   -0.1006             0.0426         0.3559
R-Squared (Between)                   0.7876             0.5767         0.6179
R-Squared (Overall)                   0.7862             0.5731         0.6172
F-statistic                           223.58             93.242         1318.2
P-value (F-stat)                      0.0000        

In [17]:
exog_vars = ['affected_after_toas_ratio','ln_res',"vow_agr_weight","vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=True,time_effects=True)
res_fe_1 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas_ratio','ln_res',"vow_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas_ratio','ln_res',"vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=True,time_effects=True)
res_fe_3 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas_ratio','ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

                                     Model Comparison                                    
                                         (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------------
Dep. Variable                         ln_vow     ln_vow_agr     ln_vow_man     ln_vow_ser
Estimator                           PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                        2304           2304           2304           2304
Cov. Est.                          Clustered      Clustered      Clustered      Clustered
R-squared                             0.2957         0.1626         0.6346         0.4988
R-Squared (Within)                    0.0179         0.3155         0.5172         0.3068
R-Squared (Between)                   0.8207         0.3902         0.6956         0.8256
R-Squared (Overall)                   0.8196         0.3898         0.6950         0.8247
F-statisti

In [18]:
exog_vars = ['affected_after_toas_ratio','ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_1 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas_ratio','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas_ratio','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_3 = mod.fit(cov_type = 'clustered',cluster_entity=False)


print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3}, stars = True, precision = "std_errors"))

                                      Model Comparison                                     
                                             (1)                  (2)                   (3)
-------------------------------------------------------------------------------------------
Dep. Variable                     ln_vow_ser_fin     ln_vow_ser_trade     ln_vow_ser_public
Estimator                               PanelOLS             PanelOLS              PanelOLS
No. Observations                            2304                 2304                  2304
Cov. Est.                              Clustered            Clustered             Clustered
R-squared                                 0.1403               0.1675                0.4346
R-Squared (Within)                        0.1163               0.1303                0.3592
R-Squared (Between)                       0.6666               0.6781                0.8344
R-Squared (Overall)                       0.6629               0.6773           

In [19]:
print(":D")

:D
